In [ ]:
import pandas as pd
import glob
from tqdm import tqdm

In [ ]:
# Folder Path on Google Drive
folder_path = '/content/drive/MyDrive/Colab/NU/IE6200/trip_data/'

# Get all CSV files in the folder
csv_files = glob.glob(folder_path + "*.csv")

# Initialize an empty list to store dataframes
dfs = []

# Loop over each file with tqdm progress bar, load it, rename columns, and append to the list
for file in tqdm(csv_files, desc="Loading and Processing CSV Files"):
    df = pd.read_csv(file)
    # Rename the columns
    df = df.rename(columns={
        'member_casual': 'rider_type',
        'rideable_type': 'bike_type'
    })
    # Append to the list of dataframes
    dfs.append(df)

# Combine all dataframes into one
combined_df = pd.concat(dfs, ignore_index=True)

# Sort the combined DataFrame by 'started_at' column
combined_df = combined_df.sort_values(by='started_at').reset_index(drop=True)

Loading and Processing CSV Files: 100%|██████████| 10/10 [00:25<00:00,  2.51s/it]


In [ ]:
# Missing values
missing_values = combined_df.isnull().sum()
missing_values

,0
ride_id,0
bike_type,0
started_at,0
ended_at,0
start_station_name,1060
start_station_id,1060
end_station_name,7026
end_station_id,7175
start_lat,0
start_lng,0


In [ ]:
# Remove rows with any missing values
combined_df = combined_df.dropna()

In [ ]:
combined_df.describe()

,start_lat,start_lng,end_lat,end_lng
count,3.761155e+06,3.761155e+06,3.761155e+06,3.761155e+06
mean,4.235856e+01,-7.108954e+01,4.235850e+01,-7.108941e+01
std,2.026244e-02,2.859435e-02,2.037073e-02,2.871729e-02
min,4.213793e+01,-7.124809e+01,4.216723e+01,-7.124776e+01
25%,4.234807e+01,-7.110707e+01,4.234807e+01,-7.110707e+01
50%,4.235848e+01,-7.109010e+01,4.235815e+01,-7.108995e+01
75%,4.236874e+01,-7.107036e+01,4.236861e+01,-7.106996e+01
max,4.253478e+01,-7.077670e+01,4.253467e+01,-7.087021e+01


In [ ]:
combined_df.head()

,ride_id,bike_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,rider_type
0,31F12D722DEC2205,classic_bike,2023-12-01 00:00:47,2023-12-01 00:18:32,MIT Carleton St at Amherst St,M32070,Lower Cambridgeport at Magazine St / Riverside Rd,M32022,42.360541,-71.086698,42.357219,-71.113872,member
1,C7B5105B0B65C0F3,classic_bike,2023-12-01 00:02:04,2023-12-01 00:07:54,MIT at Mass Ave / Amherst St,M32006,MIT Vassar St,M32042,42.358100,-71.093198,42.355601,-71.103945,member
2,68B5F2A512F98D31,classic_bike,2023-12-01 00:02:11,2023-12-01 00:03:15,South Station - 700 Atlantic Ave,A32010,Boston Landing,A32045,42.352175,-71.055547,42.356561,-71.141675,member
3,498C4A8C729ED986,classic_bike,2023-12-01 00:04:19,2023-12-01 00:14:00,One Beacon St,B32061,Columbus Ave at W. Canton St,C32077,42.358477,-71.061351,42.344742,-71.076482,member
4,2B9A06F84509E2C0,classic_bike,2023-12-01 00:04:38,2023-12-01 00:16:16,Silber Way,D32032,Dartmouth St at Newbury St,D32045,42.349495,-71.100575,42.350961,-71.077828,member


In [ ]:
# Most/least popular stations
combined_df["start_station_name"].value_counts()

,count
start_station_name,
MIT at Mass Ave / Amherst St,69503
Central Square at Mass Ave / Essex St,56334
Harvard Square at Mass Ave/ Dunster,49570
MIT Pacific St at Purrington St,39844
Charles Circle - Charles St at Cambridge St,39659
...,...
Winthrop Circle,11
Damrell st at Old Colony Ave,6
Chestnut Hill Ave. at Ledgemere Road,5


In [ ]:
# STATION NAME-to-ID VALIDATION
"""
This block of code checks for inconsistencies between station names and station IDs by identifying
cases where a station name is associated with multiple station IDs or where a station ID is
associated with multiple station names. The code does this for both start stations and end stations.
"""

# Create dictionaries to map start station names to IDs and start station IDs to names
start_name_to_ids = combined_df.groupby('start_station_name')['start_station_id'].unique().to_dict()
start_id_to_names = combined_df.groupby('start_station_id')['start_station_name'].unique().to_dict()

# Create dictionaries to map end station names to IDs and end station IDs to names
end_name_to_ids = combined_df.groupby('end_station_name')['end_station_id'].unique().to_dict()
end_id_to_names = combined_df.groupby('end_station_id')['end_station_name'].unique().to_dict()

# Check for mismatches in start stations (name to multiple IDs)
print("Start Station names mapping to multiple IDs:")
for name, ids in start_name_to_ids.items():
    if len(ids) > 1:
        print(f"{name}: {list(ids)} (Instances: {len(ids)})")

# Check for mismatches in start stations (ID to multiple names)
print("\nStart Station IDs mapping to multiple names:")
for station_id, names in start_id_to_names.items():
    if len(names) > 1:
        print(f"{station_id}: {list(names)} (Instances: {len(names)})")

# Check for mismatches in end stations (name to multiple IDs)
print("\nEnd Station names mapping to multiple IDs:")
for name, ids in end_name_to_ids.items():
    if len(ids) > 1:
        print(f"{name}: {list(ids)} (Instances: {len(ids)})")

# Check for mismatches in end stations (ID to multiple names)
print("\nEnd Station IDs mapping to multiple names:")
for station_id, names in end_id_to_names.items():
    if len(names) > 1:
        print(f"{station_id}: {list(names)} (Instances: {len(names)})")

Start Station names mapping to multiple IDs:
Somerville Hospital: ['S32020', 'S32052'] (Instances: 2)
Tremont St at Court St: ['A32046', 'A32058'] (Instances: 2)

Start Station IDs mapping to multiple names:
A32046: ['Tremont St at Court St', 'Canal St. at Causeway St.', 'Canal St at Causeway St'] (Instances: 3)
A32058: ['Tremont St. at Court St.', 'Tremont St at Court St'] (Instances: 2)
B32038: ['Chestnut Hill Ave. at Ledgemere Road', 'Chestnut Hill Ave at Ledgemere Rd'] (Instances: 2)
C32109: ['Centre St. at Allandale St.', 'Centre St at Allandale St'] (Instances: 2)
E32003: ['Hyde Square - Barbara St at Centre St', 'Hyde Square - Centre St at Perkins St'] (Instances: 2)
L32007: ['Swan Pl. at Minuteman Bikeway', 'Swan Place at Minuteman Bikeway'] (Instances: 2)
M32019: ['CambridgeSide Galleria - CambridgeSide PL at Land Blvd', 'Cambridgeside Pl at Land Blvd'] (Instances: 2)
S32052: ['Summer St at Quincy St', 'Somerville Hospital'] (Instances: 2)
V32003: ['Everett Square (Broadway at

In [ ]:
# REMOVE ROWS WITH MISMATCHES STATION NAMES AND IDS

"""
This code block identifies and removes instances from the combined_df DataFrame
where there are mismatches between station names and their corresponding IDs
for both start and end stations. Mismatches are defined as:
- Start station names mapping to multiple IDs.
- Start station IDs mapping to multiple names.
- End station names mapping to multiple IDs.
- End station IDs mapping to multiple names.
"""

# Identify problematic start station names that map to multiple IDs
problematic_start_ids = set()
for name, ids in start_name_to_ids.items():
    if len(ids) > 1:
        problematic_start_ids.update(ids)

# Identify problematic start station IDs that map to multiple names
problematic_start_names = set()
for station_id, names in start_id_to_names.items():
    if len(names) > 1:
        problematic_start_names.update(names)

# Identify problematic end station names that map to multiple IDs
problematic_end_ids = set()
for name, ids in end_name_to_ids.items():
    if len(ids) > 1:
        problematic_end_ids.update(ids)

# Identify problematic end station IDs that map to multiple names
problematic_end_names = set()
for station_id, names in end_id_to_names.items():
    if len(names) > 1:
        problematic_end_names.update(names)

# Delete rows from combined_df where start stations or end stations are problematic
combined_df_cleaned = combined_df[
    ~combined_df['start_station_id'].isin(problematic_start_ids) &
    ~combined_df['start_station_name'].isin(problematic_start_names) &
    ~combined_df['end_station_id'].isin(problematic_end_ids) &
    ~combined_df['end_station_name'].isin(problematic_end_names)
]

# Display the number of rows deleted
deleted_rows_count = combined_df.shape[0] - combined_df_cleaned.shape[0]
print(f"Number of rows deleted due to mismatches: {deleted_rows_count}")

# Update combined_df to be the cleaned DataFrame
combined_df = combined_df_cleaned

In [18]:
combined_df['end_station_id'].value_counts().get('A32058', 0)

4662

In [21]:
combined_df['start_station_name'].value_counts().get('Summer St at Quincy St', 0)

623